#### Score: 0.87571 Kagglename: M_Urban

#### Namen der Teammitglieder: Marius Urban, Ramon Jäckle, Maen Hanna

Standardcode aus Beispiel um Daten zu erhalten

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install wget

In [ ]:
import os
import pandas as pd
import random
import zipfile

In [ ]:
!python -m wget https://filr.hs-offenburg.de/filr/public-link/file-download/0dcf8b85882ae199018870bceddf437c/5426/-2254907074900866378/test_data.zip

In [ ]:
!python -m wget https://zenodo.org/record/7869954/files/products_leaflets_256.zip?download=1

In [ ]:
path_to_zip_file = 'products_leaflets_256.zip'
directory_to_extract_to = '.'
with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
    zip_ref.extractall(directory_to_extract_to)

In [ ]:
path_to_zip_file = 'test_data.zip'
directory_to_extract_to = '.'
with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
    zip_ref.extractall(directory_to_extract_to)

## Bildgröße

Für Bilder muss eine konstante Größe gewählt werden, da dies von den Netzwerken erwartet werden, aber die Bilder aus den Prospekten nicht immer gleich sind.

Die Bildgröße wurde folgendermaßen bestimmt.

1. Es wurde mit 224, 224 angefangen, da dies die Standardgröße bei den vorgefertigten Netzen aus Keras ist.
2. Es wurde schrittweise größere Größen getestet.
3. Diese zeigten eine leichte Verbesserung aber nach 256,256 kaum noch, während sich die Laufzeit verschlechterte

In [ ]:
import os
import numpy as np
from PIL import Image
import tensorflow as tf

# Verzeichnis mit den Trainingsdaten
train_directory = 'products_leaflets_256/train'
val_directory = 'products_leaflets_256/test'

# Bildgröße und Anzahl der Klassen
image_size = (256, 256)
num_classes = len(os.listdir(train_directory))

## Laden der Bilder
Als Datenstruktur für die Bilder wurden Keras Datasets gewählt. Diese bieten folgende Vorteile welche genutzt wurden:
1. automatisches Resizing der Bilder mit verschiedenen Interpolationsmöglichkeiten:
    + Die zuvor erwähnte Bildgröße mit bilinearer Interpolation wurde gewählt.
2. automatische bestimmung der Labels anhand der Ordnerstruktur
3. Das durchmischen der Daten um negative Effekte wie Overfitting zu vermeiden die auftreten, wenn Klassen immer in Gruppen nacheinander trainiert werden.
4. "Lazy-loading" in "batches" um zu verhindern, das der VRam der Grafikkarte während des Lernens voll wird.
    + Die Batchsize wurde beim Standardwert belassen, da die Batchsize keinen Einfluss auf das Lernergebnis hat, sonder nur minimale Einfluss auf die Laufzeit.

In [ ]:
# Durchlaufe das Trainingsverzeichnis und lade die Bilder
train_ds = tf.keras.utils.image_dataset_from_directory(
    train_directory,
    labels="inferred",
    label_mode="int",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=image_size,
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,)

# Durchlaufe das Validierungsverzeichnis und lade die Bilder
val_ds = tf.keras.utils.image_dataset_from_directory(
    val_directory,
    labels="inferred",
    label_mode="int",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=image_size,
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,)

In [ ]:

from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import DenseNet169
import tensorflow.keras.optimizers.legacy as legacy_optimizers
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

## Netzauswahl
Es wurden verschiedene von Keras bereitgestellte Bilderkennungsnetze getestet. Folgende wurden getestet:
+ EfficientNetV2B0
+ EfficientNetV2B1
+ ResNet50
+ ResNet101
+ MobileNetV2
+ DenseNet121
+ DenseNet169
+ DenseNet201

Desweiteren wurde ein selbstgebautes Alexnet getestet.

DenseNet169 schnitt am besten ab und wurde deshalb gewählt.

Das Netz wurde mit vortrainierten "imagenet" Gewichten genutzt.
Diese wurden fürs Training eingefroren, da selbsttrainieren das Ergebnis nur verschlechterte.
Durch Testen wurde average Pooling als beste Variante bestimmt.

In [ ]:
base_model = DenseNet169(include_top=False, pooling='avg', weights = 'imagenet', input_shape=(256, 256, 3))
base_model.layers[0].trainable = False

## Data-Augmentation
Es wurde mit verschiedenen Methoden experimentiert um Bilder mittels Data-Augmentation zu verändern um die Varianz der Bilder in den Klassen zu erhöhen.
Es wurden folgende Augmentations getestet:
1. Zufällige Rotation des Bildes
2. Zufälliges Heran-Zoomen
3. Zufälliges horizontales spiegeln
4. Zufällige Kontraständerungen

Ausnahmslos alle dieser Maßnahmen verschlechterten das Ergebnis mit der DenseNet-Familie und wurden daher verworfen.

In [ ]:
# Daten Augmentation
#data_augmentation = tf.keras.Sequential([
    #tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
    #tf.keras.layers.experimental.preprocessing.RandomZoom(0.1),
    #tf.keras.layers.RandomFlip(mode = 'horizontal'),
    #tf.keras.layers.RandomContrast(0.4)
#])

## Modelkonstruktion
Bei der Konstruktion des Gesamtmodells wurden folgende Konzepte getestet:
1. Das vom vorigen Abschnitt erwähnte und verworfene Data-Augmentation-layer
2. mehrere Denselayer am Ausgang des Modells, um zu testen ob Hinzufügen von Layern einen positiven Effekt hat.
    + Dies führte zu einer Verschlechterung
3. Ein Denselayer mit mit einer Anzahl der units die der Klassenanzahl entspricht, da das vortrainierte Netz mit 1000 Klassen trainiert wurde.

Durch das zuvor erwähnte Einfrieren der Gewichte wird nur die letzte Schicht trainiert.

In [ ]:
# Data Augmentation erweitern
model = Sequential([
    #data_augmentation,
    base_model,
    #Dense(256, activation="relu"),
    #Dense(128, activation='relu'),
    #Dense(64, activation="relu"),
    Dense(num_classes, activation='softmax')
])

## Anpassbare Lernrate
Ein wichtiger Parameter um das Ergebnis zu verbessern ist die Lernrate. Eine statische Lernrate sorgt nach einigen Epochen jedoch zu einer Verschlechterung. Eine dynamische Anpassung ist nötig. 
Hierfür beginnt man mit einer hohen Lernrate, hier wurde der Standard verwendet, und lässt eine  Metrik des Netzes beobachten, hier val_loss. Ändert sich der beobachtete Wert kaum noch über eine bestimmte Anzahl Epochen, wird die Lernrate langsam mit einem Faktor von 0.2 bis auf 1e-6 reduziert.

Diese Methode führte zur massiven Verbesserung des Ergebnises.

## Optimizer
Als Optimizer wurden Adam und SGD getestet. Während Adam schnell zu Verbesserung führte und bereits nach wenigen Epochen zu einer hohen Genauigkeit führte, lieferte SGD selbst nach 25 Epochen kein brauchbares Ergebnis. 

Somit wurde Adam als geeigneter für das verwendete Densenet169 gewählt.

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=2, min_lr=1e-6)
optimizer = Adam(learning_rate=1e-4)

## Vorzeitiger Stop
Wenn das Model über mehrere Epochen keine Verbesserung erzielt, macht ein vorzeitiges Abbrechen Sinn.

Die beobachtete Größe um die Verbesserung abzuschätzen ist der val_loss gleich wie bei der Lernratenreduzierung. Da der Value Loss minimiert werden soll ist der Modus 'min'. 
Die Anzahl Epochen ohne merkliche Veränderung bis zum Stop wurde 10 gewählt. Dies ist nicht auf ein Minimum optimiert worden, da die Gewichte der Epoche mit der besten Metrik übernommen werden.

In [ ]:
EarlyStopping_Cb = EarlyStopping(monitor = 'val_loss', mode = 'min', patience = 10, restore_best_weights = True, verbose = 1)

## Modellkompilierung

Zur Modellkompilierung kann erwähnt werden, dass die Wahl der Metriken das Ergebnis verbessern kann.

So wurde hier das Ergebnis verbessert durch indem neben der Accuracy auch der Mean-Squared Error evaluiert wurde.

## Modelltraining

Das Modell wurde in 25 Epochen trainiert. Da hier bereits das Modell in weniger Epochen bereits gestoppt wird ist die genaue Anzahl nicht wichtig im Sinne der Optimierung.

In [ ]:
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy', 'mse'])

# Trainingsparameter anpassen
batch_size = 32
epochs = 25
# Modelltraining mit erweitertem Data Augmentation
model.fit(train_ds,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=val_ds,
          callbacks=[reduce_lr, EarlyStopping_Cb])

## Prediction und speichern der Ergebnisse
Für die Prediction wurde zunächst eine Liste aller Bildnamen erstellt. Im Anschluss wurde Iterativ alle Bilder durchgegangen und in das Netz gegeben für eine Klassen-Prediction und diese in eine weitere Liste gegeben.
 Mit diesen beiden Listen wurde ein Dataframe gebildet dieser in eine CSV-Datei gespeichert.


In [ ]:
# Liste für Bildnamen und Vorhersagen
image_names = os.listdir('./test_data')
class_names = sorted(os.listdir(train_directory))
label_predictions = []
img_array = []
test_directory = 'test_data'


# Durchlaufe das Testverzeichnis und lade die Bilder
for image_name in image_names:
    url = "./test_data/" + image_name
    img = tf.keras.utils.load_img(url, target_size=image_size)
    img_array = tf.keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)
    predictions = model.predict(img_array)
    score = tf.nn.softmax(predictions[0])
    label_predictions.append(class_names[np.argmax(score)])


# Erstelle den DataFrame und speichere die Vorhersagen in einer CSV-Datei
df = pd.DataFrame(columns=['image_id', 'label'])
df['image_id'] = image_names
df['label'] = label_predictions
df.to_csv('submission.csv', index=False)

## Ergebnisanalyse
Folgende Erkenntnisse konnten bei den Versuchen gewonnen werden:
+ Verbesserungsansätzte müssen für jedes Netz seperat getestet werden. So zeigte sich eine Verbesserung des Ergebnisses mittels Dataaugmentation beim EfficientNetV2B0. Bei der DenseNet-Familie verschlechterten sich jedoch die Ergebnisse.
+ Eine Anpassung der Lernrate erzielte bei jedem getesteten Netz, eine beträchtliche Verbesserung.
+ Mit vortrainierten Netzen können sehr schnell gute Ergebnisse erzielt werden.



Folgende Aspekte können an diesem Ansatz verbessert werden:
+ Zusätzlich zur Produkterkennung eine Texterkennung implementieren und die Ansätze fusionieren
+ Bildvorverarbeitungtechniken testen z.b Kontrast aller Bilder erhöhen
+ Parameter anpassen die bisher noch nicht angepasst wurden. Beispielsweise wurden die Lerrnraten beim Optimizer und der Lernratenreduzierung beim Versuch mit dem Densenet169 nicht angepasst sondern mit Werten aus Test mit anderen Netzten übernommen.
+ Zusätztliche Ansätze zur Optimierung könnten recherchiert werden und getestet werden.